### Load required packages

The following commands will load the required packages to execute the script in your IDE. If you are running this script in Excel, you don't need to install these packages with the command pip (see README file). All of them are already installed by default (numpy, pandas, scikit-learn, statsmodels, matplotlib and warning).

However you do need to initalize some of them in your script in Excel. In essence, you need to replace the script to:

- from statsmodels.tsa.statespace.sarimax import SARIMAX
- from sklearn.base import BaseEstimator, RegressorMixin
- from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

Note: For further information of preloaded libraries in Excel click: Formulas -> Initialization

In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import matplotlib.pyplot as plt
import warnings

In [ ]:
warnings.filterwarnings("ignore")

### Load and transform the data

In Excel, the following script will load the data on the address "B6:B41" as create a Dataframe. Time series model usually works with one-dimension dataset. The command data.to_numpy().ravel() will transform the dataframe "data" to a ndarray named "y_series"

In [ ]:
data = xl("Analysis!B6:B41")
y_series = data.to_numpy().ravel()
X_dummy = np.zeros((len(y_series), 1))

### Create functions for SARIMA model

The follow script defines a scikit-learn–style class for SARIMA modeling, with three key methods: initialization (__init__), fitting (fit), and prediction (predict).

In [ ]:
class SARIMAModel(BaseEstimator, RegressorMixin):
    def __init__(self, order=(0,0,0), seasonal_order=(0,0,0,0)):
        self.order = order
        self.seasonal_order = seasonal_order
        self.model_ = None

    def fit(self, X, y):
        y = np.asarray(y).ravel()
        self.model_ = SARIMAX(
            y,
            order=self.order,
            seasonal_order=self.seasonal_order,
            enforce_stationarity=True,
            enforce_invertibility=True
        ).fit(disp=False)
        return self

    def predict(self, X):
        start = len(self.model_.data.endog)
        end = start + len(X) - 1
        forecast = self.model_.predict(start=start, end=end)
        return forecast

### Grid search parameters
The following script creates the grid search parameters. 

Note: Given that in statsmodels "auto-arima" detection is not possible, the alternative is to execute the sarima model by using several order x seasonal order combinations and, then, finding the best parameters ( with lowest error). 


In [5]:
param_grid = {
    "order": [(1,0,0), (0,1,0), (0,0,1), (1,1,0), (1,0,1), (1,1,1), (2,0,0), (2,1,0), (2,0,1), (2,1,1)],
    "seasonal_order": [(0,0,0,0), (0,0,0,12), (1,0,0,12), (0,1,0,12), (0,0,1,12), (1,1,0,12), (1,0,1,12), (0,1,1,12), (1,1,1,12)]
}

### Define the grid search hyper-parameters

The command GridSearchCV() from scikit-learn exhaustive search over specified parameter values for an estimator. The following script defines the hyper-parameters for the grid search by using SARIMA model, param_grid, cross validation and score.

In [ ]:
grid_search = GridSearchCV(
    SARIMAModel(),
    param_grid,
    cv=TimeSeriesSplit(n_splits=2),
    scoring="neg_mean_squared_error"
)

And executes the method fit in the grid_search

In [ ]:
grid_search.fit(X_dummy, y_series)

The following script identify the best estimator based on score results, and creates 3 objects:
- forecast (for the next 12 periods)
- fited_values (fitted values of historical data, also called "p hat")
- forecast_ci = containing the confidence interval of the forecasts

In [ ]:
best_model = grid_search.best_estimator_
forecast = np.round(best_model.predict(np.zeros((12, 1))))
fited_values = best_model.model_.fittedvalues
forecast_ci = best_model.model_.get_forecast(steps=12).conf_int(alpha=0.05)

The follow script plots the data

In [ ]:
plot_data_actual = pd.DataFrame({  'Sales': y_series,
                            'Fitted': fited_values,

                        })
plot_data_forecast = pd.DataFrame({
                            'Forecast': forecast,
                            'Lower CI': forecast_ci[:, 0],
                            'Upper CI': forecast_ci[:, 1],})                           


plot_data_forecast['Lower CI'] = plot_data_forecast['Lower CI'].clip(lower=0)
plot_data_forecast['Upper CI'] = plot_data_forecast['Upper CI'].clip(upper=plot_data_forecast['Forecast'].max() * 1.5)

plt.figure(figsize=(12, 6)) 
plt.plot(plot_data_actual['Sales'], label='Sales', color='blue') 
plt.plot(plot_data_actual['Fitted'], label='Fitted', color='orange')
plt.plot(range(len(y_series), len(y_series) + len(forecast)), forecast, label='Forecast', color='green', marker='o')
plt.fill_between(plot_data_forecast.index + len(y_series), plot_data_forecast['Lower CI'], plot_data_forecast['Upper CI'], color='lightgreen', alpha=0.3, label='Forecast 95% CI')
plt.axvline(x=len(y_series)-1, color='red', linestyle='--', label='Forecast Start') 
plt.xlabel('Time') 
plt.ylabel('Sales') 
plt.title(f'Sales Forecast using SARIMA {best_model.order} x {best_model.seasonal_order}') 
plt.legend(loc='lower right') 
plt.show()